In [ ]:
from generation.implementation_suggestion import NoSuggestion

generation_results = "results/generation"
collection_name = "books_test"
sequence = 0

suggestion = NoSuggestion()
suggestion.save_suggestion(f"{generation_results}/{collection_name}/test{sequence}.json")

In [2]:
from vectorization.semantic_vector_database import SemanticVectorDatabase
from llm_providers.llm_provider import LLMExecutor
from generation.semantic_data_retriever import SemanticDataRetriever
from generation.code_sense import CodeSenseImplementationGenerator

# Test requests
test_requests = [
    "I want to add a new rule to calculate loyalty points",
    "Create a new loyalty point reward for VIP customers",
    "Add validation for minimum order amount in loyalty rules",
    "Implement a service to check customer loyalty status",
    "Add integration with payment service for loyalty points"
]

generation_results = "results/generation"
def run_generator(vector: SemanticVectorDatabase, collection_name: str, provider_to_test: LLMExecutor):
    """Demo function to test implementation generation"""

    collection = vector.get_collection(collection_name)
    retriever = SemanticDataRetriever(collection)

    generator = CodeSenseImplementationGenerator(retriever, provider_to_test)

    for sequence, request in enumerate(test_requests):
        print(f"\n{'='*80}")
        print(f"TESTING REQUEST: {request}")
        print(f"{'='*80}")

        try:
            # Generate suggestion
            suggestion = generator.generate_complete_suggestion(
                user_request=request
            )

            # Display results
            suggestion.display_suggestion()

            # Save suggestion
            suggestion.save_suggestion(f"{generation_results}/{collection_name}/test{sequence}.json")

        except Exception as e:
            print(f"Error testing {provider_to_test}: {e}")

In [ ]:
import os
from dotenv import load_dotenv
from llm_providers.anthropic import AnthropicExecutor
from llm_providers.ollama import OllamaExecutor
from llm_providers.openai import OpenAIExecutor
from vectorization.semantic_vector_database import SemanticVectorDatabase

load_dotenv()

vector_results = "results/vectorization"
rag_results = "results/rag"

chroma_base = "{model}/loyalty_points_kb"

loyalty_collection = "loyalty_code_semantics_{llm}"
anthropic = AnthropicExecutor(os.getenv("ANTHROPIC_API_KEY"))
ollama = OllamaExecutor()
openai = OpenAIExecutor(os.getenv("OPENAI_API_KEY"))

models = [
    {
        "model": "all-MiniLM-L6-v2",
        "llms": ["claude3.5", "codellama", "gpt4.1"],
        "providers": [anthropic, ollama, openai]
    },
    {
        "model": "all-mpnet-base-v2",
        "llms": ["claude3.5", "codellama", "gpt4.1"],
        "providers": [anthropic, ollama, openai]
    }
]

rag_testers = []
reports = []

for model in models:
    embedding_model = model["model"]
    print("embedding_model: ", embedding_model)
    db_path = f"{vector_results}/{chroma_base.format(model=embedding_model)}"
    print("db_path: ", db_path)

    # Fix: Use enumerate() to get both index and value
    for i, llm in enumerate(model["llms"]):
        # Fix: Use the llm string directly since it's already a string
        collection_name = loyalty_collection.format(llm=llm)
        provider = model["providers"][i]

        vector_db = SemanticVectorDatabase(db_path, embedding_model)
        run_generator(vector_db, collection_name, provider)
